# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/martin/.pyenv/shims/python (found version "3.6.5") 
-- Found PythonLibs: /home/martin/.pyenv/versions/3.6.5/lib/libpython3.6m.a
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/martin/Desktop/repos/metnum-tp2-20191c
-- Configuring 

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home/martin/Desktop/repos/metnum-tp2-20191c/notebooks
Python 3.6.5


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

./._imdb_small.csv
imdb_small.csv
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,By far this has to be one of the worst movies ...,neg,7787_10.txt
freq,6275,2,6322,2


In [5]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [21]:
import sentiment

clf = sentiment.KNNClassifier(100)
#esta linea y el siguiente bloque son para correr KNN sin PCA
#esta comentado para poder ejecutar todos los bloques sin ejecutar este
#por equivocacion
#clf.fit(X_train, y_train)


In [22]:
#%%time
#from sklearn.metrics import accuracy_score
#y_pred = clf.predict(X_test)

#acc = accuracy_score(y_test, y_pred)
#print("Accuracy: {}".format(acc))

In [23]:
pca = sentiment.PCA(30)

In [31]:
pca.fit(X_train,10000, 0.000000001)

In [25]:
pca_X_train = pca.transform(X_train, 5)
pca_X_test = pca.transform(X_test,5)

In [26]:
print(pca_X_test.shape[1])
print(pca_X_test)

5
[[ -6.90497268  -2.81010477  -0.93969949  -2.6381538   -2.85025472]
 [ -7.81907144   1.4215357   -1.45152066   2.06105015   0.62981852]
 [ -3.26337631  -1.48763321  -0.44734354  -0.95986524   1.96630703]
 ...
 [-18.35496461  -0.47617457  -2.7777734    4.01547945   3.0803519 ]
 [ -6.3272365   -0.23604173   1.11981217   2.20128377  -0.3660268 ]
 [-18.26258536   0.67115043  -1.17023443  -2.0891063    5.85577905]]


In [27]:
%%time
clf.fit(pca_X_train, y_train)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

y_pred = clf.predict(pca_X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))



Accuracy: 0.5655776892430279
CPU times: user 4.02 s, sys: 7.99 ms, total: 4.03 s
Wall time: 4.01 s


In [10]:
#All in one. Thuis cell defines the tester functions.
import time
from sklearn.feature_extraction.text import CountVectorizer
def test(max_alpha_, min_alpha_, alpha_step_, max_df_=0.90, min_df_=0.01, max_features_=5000, knn_neighbours_=100, log=False):

    vectorizer = CountVectorizer(max_df=max_df_, min_df=min_df_, max_features=max_features_)
    vectorizer.fit(text_train)

    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

    import sentiment

    clf = sentiment.KNNClassifier(knn_neighbours_)
    pca = sentiment.PCA(max_alpha_)
    #fitteo con el x_train calculado arriba
    pca.fit(X_train,10000, 0.0000000001)
    #loopeo para todos los alphas
    for alpha_ in range(min_alpha_, max_alpha_, alpha_step_):
        pca_X_train = pca.transform(X_train, alpha_)
        pca_X_test = pca.transform(X_test,alpha_)

        #Timing starts, same as previous cell:
        start = time.time()

        clf.fit(pca_X_train, y_train)
        from sklearn.metrics import accuracy_score
        from sklearn.metrics import precision_score
        from sklearn.metrics import recall_score

        y_pred = clf.predict(pca_X_test)

        acc = accuracy_score(y_test, y_pred)
        pre = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = 2*(pre*rec)/(pre+rec)
        #Timing ends, again, like in previous cell.
        end = time.time()
        delta_time = end - start
        delta_time_str = str(round(delta_time, 2))
        if log:
            comma = ", "
            f = open("../data/test_results/out_df_1.csv","a+")
            f.write("\n")
            f.write(str(round(acc,2)) + comma)
            f.write(str(round(pre,2)) + comma)
            f.write(str(round(rec,2)) + comma)
            f.write(delta_time_str + comma)
            f.write(str(round(max_df_,2)) + comma)
            f.write(str(round(min_df_,2)) + comma)
            f.write(str(round(max_features_,2)) + comma)
            f.write(str(round(knn_neighbours_,2)) + comma)
            f.write(str(round(alpha_,2))+ comma)
            f.write(str(round(f1,2)))
            f.close()

    return f1


In [ ]:
#Actual testing, this cell runs the tester functions and logs or prints results.
#It may take a long time to run.
import numpy as np
#OJO, puse esto en true para testear
test_individually = False
if test_individually:
    test(max_alpha_=11, min_alpha_= 10, alpha_step_=2,max_df_=0.90, min_df_=0.01, max_features_=5000, knn_neighbours_=10,log=True)
else:
    max_alpha = 101
    min_alpha = 100
    alpha_step = 1
    max_features = 5000
    knn_neighbours = 50
    #for min_df in np.arange(0.01, 0.49,0.03):
    for min_df in np.arange(0.01, 0.06,0.03):
        for max_df in np.arange(0.49, 0.99,0.03):
            test(max_alpha_= max_alpha, min_alpha_= min_alpha, alpha_step_=alpha_step,max_df_= max_df, min_df_= min_df, max_features_=max_features, knn_neighbours_=knn_neighbours,log=True)


In [47]:
def gradient_descent(pca, p0, start_value, max_iters, debug = False):
    
    old_val = start_value
    p1 = [p0[0], p0[1], p0[2], p0[3], p0[4]]
    aggr = 1
    aggr_max = 3
    log = True

    while True:
        
        p0 = p1
        print("Current vector: " + str(p0))
        
        step0 = aggr*p0[0]*0.05#0.03
        step1 = aggr*p0[1]*0.05 #0.03
        step2 = int(aggr*p0[2]*0.5) #1000
        step3 = int(aggr*p0[3]*0.5) #10
        step4 = int(aggr*p0[4]*0.5) #100

        changed = False
        
        if max_iters == 0:
            print("Ending descent due to max_iters")
            break
        max_iters -= 1
        
        next_val = test(pca, max(0,p0[0]+step0), p0[1], p0[2], p0[3], p0[4], log)
        if next_val > old_val:
            p1 = [p0[0]+step0, p0[1], p0[2], p0[3], p0[4]]
            old_val = next_val
            changed = True

        next_val = test(pca, max(0,p0[0]-step0), p0[1], p0[2], p0[3], p0[4], log)
        if next_val > old_val:
            p1 = [p0[0]-step0, p0[1], p0[2], p0[3], p0[4]]
            old_val = next_val
            changed = True

        next_val = test(pca, p0[0], max(0,p0[1]+step1), p0[2], p0[3], p0[4], log)
        if next_val > old_val:
            p1 = [p0[0], p0[1]+step1, p0[2], p0[3], p0[4]]
            old_val = next_val
            changed = True
            
        next_val = test(pca, p0[0], max(0,p0[1]-step1), p0[2], p0[3], p0[4], log)
        if next_val > old_val:
            p1 = [p0[0], p0[1]-step1, p0[2], p0[3], p0[4]]
            old_val = next_val
            changed = True
        
        next_val = test(pca, p0[0], p0[1], max(3,p0[2]+step2), p0[3], p0[4], log)
        if next_val > old_val:
            p1 = [p0[0], p0[1], p0[2]+step2, p0[3], p0[4]]
            old_val = next_val
            changed = True
            
        next_val = test(pca, p0[0], p0[1], max(3,p0[2]-step2), p0[3], p0[4], log)
        if next_val > old_val:
            p1 = [p0[0], p0[1], p0[2]-step2, p0[3], p0[4]]
            old_val = next_val
            changed = True

        next_val = test(pca, p0[0], p0[1], p0[2], max(3,p0[3]+step3), p0[4], log)
        if next_val > old_val:
            p1 = [p0[0], p0[1], p0[2], p0[3]+step3, p0[4]]
            old_val = next_val
            changed = True
            
        next_val = test(pca, p0[0], p0[1], p0[2], max(3,p0[3]-step3), p0[4], log)
        if next_val > old_val:
            p1 = [p0[0], p0[1], p0[2], p0[3]-step3, p0[4]]
            old_val = next_val
            changed = True

        next_val = test(pca, p0[0], p0[1], p0[2], p0[3], max(3,p0[4]+step4), log)
        if next_val > old_val:
            p1 = [p0[0], p0[1], p0[2], p0[3], p0[4]+step4]
            old_val = next_val
            changed = True

        next_val = test(pca, p0[0], p0[1], p0[2], p0[3], max(3,p0[4]-step4), log)
        if next_val > old_val:
            p1 = [p0[0], p0[1], p0[2], p0[3], p0[4]-step4]
            old_val = next_val
            changed = True

        if not changed:
            aggr *= 1.15
            if debug:
                print("Local maximum found (" + str(old_val) + "), rising aggressiveness to: "+ str(aggr))
                print("Argument vector: " + str(p1))
            if aggr > aggr_max:
                aggr = 1
                print("Aggressiveness got to high, resetting to one.")
            
        if debug and changed:
            print("Found a better value: " + str(old_val))
            print("New argument vector: " + str(p1) + "\n")
            
        
p0 = [0.90, 0.01,3000, 20, 30]
p0 = [0.9, 0.01, 3000, 165, 275]
p0 = [0.9, 0.01, 1500, 124, 618]
v0 = test(pca, p0[0], p0[1], p0[2], p0[3], p0[4], log=True)
gradient_descent(pca, p0, v0, max_iters = 1000000, debug = True)


Current vector: [0.9, 0.01, 3000, 165, 275]
Found a better value: 0.68652522130111
New argument vector: [0.9, 0.01, 1500, 165, 275]

Current vector: [0.9, 0.01, 1500, 165, 275]
Found a better value: 0.7004207954391203
New argument vector: [0.9, 0.01, 1500, 165, 412]

Current vector: [0.9, 0.01, 1500, 165, 412]
Found a better value: 0.7096691129136089
New argument vector: [0.9, 0.01, 1500, 165, 618]

Current vector: [0.9, 0.01, 1500, 165, 618]
Found a better value: 0.7109102646868949
New argument vector: [0.9, 0.01, 1500, 83, 618]

Current vector: [0.9, 0.01, 1500, 83, 618]
Found a better value: 0.7128278000258365
New argument vector: [0.9, 0.01, 1500, 124, 618]

Current vector: [0.9, 0.01, 1500, 124, 618]
Local maximum found (0.7128278000258365), rising aggressiveness to: 1.15
Argument vector: [0.9, 0.01, 1500, 124, 618]
Current vector: [0.9, 0.01, 1500, 124, 618]


KeyboardInterrupt: 